# 23 novembre 2021. Correction bug sur la lecture des donnees MEC

# 19 juillet 2021. Remplacement des données du champ électrique par données GSE

Les données du champ électrique sont à présent obtenues via les fichiers GSE "mms#_edp_dce_gse_brst_l2"

# 16 novembre 2020. Ajout de la lecture des fonctions de distribution des particules
sur le modèle du code de Roberto

# 12 octobre 2020. Corrections Laurence

Les données de positions sont maintenant récupérées dans les fichiers MEC (Magnetic Ephemeris Coordinates) et plus dans les fichiers FGM (mais j'ai laissé l'ancienne versiond dans le code). J'ai aussi coupé le code en tranches pour qu'on voit mieux ce qu'on fait, étape par étape : lecture des données, puis interpolation.Enfin l'interpolation des positions est quadratique pour éviter des sauts dans la dérivée de la position.

Ancien commentaire :
Il faut ranger ensuite les fichiers dans le bon dossier ! Les fichiers sont créés dans le dossier où est ce NB (analyse_1sc). Il faut les ranger dans C:\Users\rezeau\Documents\Laurence\recherche\data\txt_tables\20151016\130524_43 ou similaire.

-) txt file creator for MMS data (time, B, Ni, Ui, Ne, Ue, Ti, Te, E, J)
-) all data are interpolated to the MMS1's FGM time array (128 S/s)
-) instructions: write in the "user's choice" section the start and the end dates in the format: year, mounth, day, hour, minute and second

Version modifiée par Jean Le Hénaff juillet 2020 pour faire en sorte que l'échelle de temps soit régulièrement espacée

# 9 octobre 2020
Message d'Alexis Jeandet à propos de l'utilisation de cdas:
En général, j'utilise soit le site web avec leur formulaire pour retrouver les noms, soit depuis python, tu peux utiliser des expressions régulières pour chercher les couples dataset/varaibles que tu cherches:

dans une cellule de notebook:

cdas.get_datasets('sp_phys',idPattern='MMS1.*MEC.*SRVY.*',)

Ca va te lister tous les datasets qui matchent la regex 'MMS1.*MEC.*SRVY.*'.

Puis une fois que tu as trouvé le dataset pour avoir le nom des variables:

cdas.get_variables('sp_phys', 'MMS1_MEC_SRVY_L2_EPHTS04D')['VariableDescription']

In [1]:
#_user's choice

# Select frequency. /!\ Should be 128Hz since this script looks for burst mode files. /!\
# Important : 1/frequency should be representable in floating-point arithmetic. ie. frequency = 2^a * 5^b
frequency = 128

#préliminaires
from ai import cdas
import datetime
import matplotlib.pyplot as plt
import numpy as np
from scipy.interpolate import interp1d
import pandas as pd

#Nouvels importations
from matplotlib.colors import LogNorm 
from mpl_toolkits.axes_grid1 import *
from mpl_toolkits.axisartist import *
from scipy.ndimage import gaussian_filter1d
#Commandes magiques pour les figures
import matplotlib.pyplot as plt
import os

%matplotlib widget

from scipy import interpolate
from scipy import integrate
def datetosec(timearray,time_zero):
    [Y,M,D,h,m,s] = time_zero
    DT = [obj-datetime.datetime(Y,M,D,h,m,s) for obj in timearray]
    time = np.array([dt.microseconds * 1e-6 + dt.seconds + dt.days * 86400 for dt in DT])
    return(time)

kmtom = 10**3
qe = 1.6 * 10**(-19)
cm3tom3 = 10**6 
dti=0.150



def interpolation_pressure(time_B, time_P, P):
    P=P[np.where(np.abs(time_P-(datetime_end-datetime_init).total_seconds()/2.)<(datetime_end-datetime_init).total_seconds()/2.)]
    time_P=time_P[np.where(np.abs(time_P-(datetime_end-datetime_init).total_seconds()/2.)<(datetime_end-datetime_init).total_seconds()/2.)]

    N_Pi1=len(time_P)
    Pi1_init=np.reshape(np.transpose(P*1.e-9) , (3,3,N_Pi1))

    dtPi1_init=np.zeros((3,3,N_Pi1))
    dtPi1_smooth=np.zeros((3,3,N_Pi1))
    N=len(time_B)
    dtPi1=np.zeros((3,3,N))
    Pi1=np.zeros((3,3,N))
    for i in range(3):
        for j in range(3):
            dtPi1_init[i,j,:] = np.gradient(Pi1_init[i,j,:], 0.150)
            dtPi1_smooth[i,j,:] = gaussian_filter1d(dtPi1_init[i,j,:], 1.20)  #GIULIO guarda come ottimizzare qua
            f_sur=interpolate.interp1d(time_P, dtPi1_smooth[i,j,:], kind='cubic', fill_value='extrapolate')
            dtPi1[i,j,:] =  f_sur(time_B)
            for t in range(1,N):
                Pi1[i,j,t] = integrate.simpson(dtPi1[i,j,0:t], time_B[0:t])
            Pi1[i,j,:] = Pi1[i,j,:]  - np.mean(Pi1[i,j,:]) + np.mean(Pi1_init[i,j,:])

    return Pi1

def interpolation_vn(time_B, time, qnt):
    qnt=qnt[np.where(np.abs(time-(datetime_end-datetime_init).total_seconds()/2.)<(datetime_end-datetime_init).total_seconds()/2.)]
    time=time[np.where(np.abs(time-(datetime_end-datetime_init).total_seconds()/2.)<(datetime_end-datetime_init).total_seconds()/2.)]


    N_1=len(time)
    dt1_init=np.zeros(N_1)
    dt1_smooth=np.zeros(N_1)
    N=len(time_B)
    dt1=np.zeros(N)
    qnt1=np.zeros(N)
    dt1_init = np.gradient(qnt, 0.150)
    dt1_smooth = gaussian_filter1d(dt1_init, 1.20)  #GIULIO guarda come ottimizzare qua
    f_sur=interpolate.interp1d(time, dt1_smooth, kind='cubic', fill_value='extrapolate')
    dt1 =  f_sur(time_B)
    for t in range(1,N):
        qnt1[t] = integrate.simpson(dt1[0:t], time_B[0:t])
    qnt1 = qnt1  - np.mean(qnt1) + np.mean(qnt)
    return qnt1

def interpolation_vn_GIU(time_B, time, qnt):
    qnt=qnt[np.where(np.abs(time-(datetime_end-datetime_init).total_seconds()/2.)<(datetime_end-datetime_init).total_seconds()/2.)]
    time=time[np.where(np.abs(time-(datetime_end-datetime_init).total_seconds()/2.)<(datetime_end-datetime_init).total_seconds()/2.)]
    N_1=len(time)
    Nw = 1
    qnt_datasmooth=np.zeros((N_1))
    for t in range(N_1):
        minw = max(t-Nw, 0)
        maxw = min(t+Nw+1, N_1)
        wrange=np.arange(minw,maxw)
        qnt_datasmooth[t]=np.mean(qnt[wrange])
    
    N=len(time_B)
    qnt_smooth=np.zeros((N))
    f_sur=interpolate.interp1d(time, qnt_datasmooth, kind='cubic', fill_value='extrapolate')
    qnt_smooth =  f_sur(time_B)    

    return qnt_smooth

def interpolation_pressure_GIU(time_B, time_P, P):
    P=P[np.where(np.abs(time_P-(datetime_end-datetime_init).total_seconds()/2.)<(datetime_end-datetime_init).total_seconds()/2.)]
    time_P=time_P[np.where(np.abs(time_P-(datetime_end-datetime_init).total_seconds()/2.)<(datetime_end-datetime_init).total_seconds()/2.)]

    N_Pi1=len(time_P)
    Pi1_init=np.reshape(np.transpose(P*1.e-9) , (3,3,N_Pi1))
    Nw = 1
    Pi1_datasmooth=np.zeros((3,3,N_Pi1))
    for t in range(N_Pi1):
        minw = max(t-Nw, 0)
        maxw = min(t+Nw+1, N_Pi1)
        wrange=np.arange(minw,maxw)
        Pi1_datasmooth[...,t]=np.mean(Pi1_init[...,wrange],axis=2)
    
    N=len(time_B)
    Pi1_smooth=np.zeros((3,3,N))
    for i in range(3):
        for j in range(3):
            f_sur=interpolate.interp1d(time_P, Pi1_datasmooth[i,j,:], kind='cubic', fill_value='extrapolate')
            Pi1_smooth[i,j,:] =  f_sur(time_B)    

    return Pi1_smooth

In [2]:
#_downloading data : ajout des données MEC octobre 2020
directory_array=np.loadtxt("/home/ballerini/Desktop/NEWstudy/data/txt_tables/folder.txt", dtype=np.str)

for direct in [0]:#range(126,len(directory_array)):
    day = str(directory_array[direct])[2:10]
    hour = str(directory_array[direct])[11:21]
    print(day, hour)
    time_init=[int(int(day)/1e4), int(int(day)%1e4/1e2), int(int(day)%1e2),int(hour[0:2]), int(hour[2:4]), int(hour[4:6])]
    time_init=[2015, 12, 28, 22, 11, 57]
    print(time_init)
    datetime_init = datetime.datetime(time_init[0],time_init[1],time_init[2],time_init[3],time_init[4],time_init[5])
    delta=datetime.timedelta(seconds=int(hour[7:len(hour)]))
    delta=datetime.timedelta(seconds=int(20))

    datetime_end =datetime_init+delta
    print(datetime_init, datetime_end)

    #_download period
    init = datetime_init - datetime.timedelta(minutes=1)
    end = datetime_end + datetime.timedelta(minutes=1) 


    data_MMS1_FGM = cdas.get_data('sp_phys', 'MMS1_FGM_BRST_L2', init, end, ['mms1_fgm_r_gse_brst_l2','mms1_fgm_b_gse_brst_l2'], cdf=True)
    data_MMS2_FGM = cdas.get_data('sp_phys', 'MMS2_FGM_BRST_L2', init, end, ['mms2_fgm_r_gse_brst_l2','mms2_fgm_b_gse_brst_l2'], cdf=True)
    data_MMS3_FGM = cdas.get_data('sp_phys', 'MMS3_FGM_BRST_L2', init, end, ['mms3_fgm_r_gse_brst_l2','mms3_fgm_b_gse_brst_l2'], cdf=True)
    data_MMS4_FGM = cdas.get_data('sp_phys', 'MMS4_FGM_BRST_L2', init, end, ['mms4_fgm_r_gse_brst_l2','mms4_fgm_b_gse_brst_l2'], cdf=True)
    print('FGM data read')



    #modification LR 16 novembre 2020 ajout spectrogrammes

    data_MMS1_FPI_i = cdas.get_data('sp_phys', 'MMS1_FPI_BRST_L2_DIS-MOMS', init, end, ['mms1_dis_numberdensity_brst','mms1_dis_bulkv_gse_brst','mms1_dis_temptensor_gse_brst','mms1_dis_energyspectr_omni_brst'], cdf=True)
    data_MMS2_FPI_i = cdas.get_data('sp_phys', 'MMS2_FPI_BRST_L2_DIS-MOMS', init, end, ['mms2_dis_numberdensity_brst','mms2_dis_bulkv_gse_brst','mms2_dis_temptensor_gse_brst','mms2_dis_energyspectr_omni_brst'], cdf=True)
    data_MMS3_FPI_i = cdas.get_data('sp_phys', 'MMS3_FPI_BRST_L2_DIS-MOMS', init, end, ['mms3_dis_numberdensity_brst','mms3_dis_bulkv_gse_brst','mms3_dis_temptensor_gse_brst','mms3_dis_energyspectr_omni_brst'], cdf=True)
    data_MMS4_FPI_i = cdas.get_data('sp_phys', 'MMS4_FPI_BRST_L2_DIS-MOMS', init, end, ['mms4_dis_numberdensity_brst','mms4_dis_bulkv_gse_brst','mms4_dis_temptensor_gse_brst','mms4_dis_energyspectr_omni_brst'], cdf=True)

    data_MMS1_FPI_e = cdas.get_data('sp_phys', 'MMS1_FPI_BRST_L2_DES-MOMS', init, end, ['mms1_des_numberdensity_brst','mms1_des_bulkv_gse_brst','mms1_des_temptensor_gse_brst','mms1_des_energyspectr_omni_brst'], cdf=True)
    data_MMS2_FPI_e = cdas.get_data('sp_phys', 'MMS2_FPI_BRST_L2_DES-MOMS', init, end, ['mms2_des_numberdensity_brst','mms2_des_bulkv_gse_brst','mms2_des_temptensor_gse_brst','mms2_des_energyspectr_omni_brst'], cdf=True)
    data_MMS3_FPI_e = cdas.get_data('sp_phys', 'MMS3_FPI_BRST_L2_DES-MOMS', init, end, ['mms3_des_numberdensity_brst','mms3_des_bulkv_gse_brst','mms3_des_temptensor_gse_brst','mms3_des_energyspectr_omni_brst'], cdf=True)
    data_MMS4_FPI_e = cdas.get_data('sp_phys', 'MMS4_FPI_BRST_L2_DES-MOMS', init, end, ['mms4_des_numberdensity_brst','mms4_des_bulkv_gse_brst','mms4_des_temptensor_gse_brst','mms4_des_energyspectr_omni_brst'], cdf=True)



    print('FPI data read')

    data_MMS1_EDP_E = cdas.get_data('sp_phys', 'MMS1_EDP_BRST_L2_DCE', init, end, ['mms1_edp_dce_gse_brst_l2'], cdf=True)
    data_MMS2_EDP_E = cdas.get_data('sp_phys', 'MMS2_EDP_BRST_L2_DCE', init, end, ['mms2_edp_dce_gse_brst_l2'], cdf=True)
    data_MMS3_EDP_E = cdas.get_data('sp_phys', 'MMS3_EDP_BRST_L2_DCE', init, end, ['mms3_edp_dce_gse_brst_l2'], cdf=True)
    data_MMS4_EDP_E = cdas.get_data('sp_phys', 'MMS4_EDP_BRST_L2_DCE', init, end, ['mms4_edp_dce_gse_brst_l2'], cdf=True)
    print('E data read')

    import speasy as spz

    MMS1_Pi = spz.get_data("amda/mms1_brst_dispres",init, end)
    MMS2_Pi = spz.get_data("amda/mms2_brst_dispres",init, end)
    MMS3_Pi = spz.get_data("amda/mms3_brst_dispres",init, end)
    MMS4_Pi = spz.get_data("amda/mms4_brst_dispres",init, end)
    print('Pi data read') 

    #lecture des fichiers MEC
    if (time_init[2]!=31):
        data_MMS1_MEC= cdas.get_data('sp_phys', 'MMS1_MEC_SRVY_L2_EPHTS04D',  datetime.datetime(time_init[0], time_init[1], time_init[2], 0, 0, 0), datetime.datetime(time_init[0], time_init[1], time_init[2]+1, 0, 0, 0), ['mms1_mec_r_gse'], cdf=True)
        data_MMS2_MEC= cdas.get_data('sp_phys', 'MMS2_MEC_SRVY_L2_EPHTS04D',  datetime.datetime(time_init[0], time_init[1], time_init[2], 0, 0, 0), datetime.datetime(time_init[0], time_init[1], time_init[2]+1, 0, 0, 0), ['mms2_mec_r_gse'], cdf=True)
        data_MMS3_MEC= cdas.get_data('sp_phys', 'MMS3_MEC_SRVY_L2_EPHTS04D',  datetime.datetime(time_init[0], time_init[1], time_init[2], 0, 0, 0), datetime.datetime(time_init[0], time_init[1], time_init[2]+1, 0, 0, 0), ['mms3_mec_r_gse'], cdf=True)
        data_MMS4_MEC= cdas.get_data('sp_phys', 'MMS4_MEC_SRVY_L2_EPHTS04D',  datetime.datetime(time_init[0], time_init[1], time_init[2], 0, 0, 0), datetime.datetime(time_init[0], time_init[1], time_init[2]+1, 0, 0, 0), ['mms4_mec_r_gse'], cdf=True)
    else:
        data_MMS1_MEC= cdas.get_data('sp_phys', 'MMS1_MEC_SRVY_L2_EPHTS04D',  datetime.datetime(time_init[0], time_init[1], time_init[2], 0, 0, 0), datetime.datetime(time_init[0], time_init[1]+1, 1, 0, 0, 0), ['mms1_mec_r_gse'], cdf=True)
        data_MMS2_MEC= cdas.get_data('sp_phys', 'MMS2_MEC_SRVY_L2_EPHTS04D',  datetime.datetime(time_init[0], time_init[1], time_init[2], 0, 0, 0), datetime.datetime(time_init[0], time_init[1]+1, 1, 0, 0, 0), ['mms2_mec_r_gse'], cdf=True)
        data_MMS3_MEC= cdas.get_data('sp_phys', 'MMS3_MEC_SRVY_L2_EPHTS04D',  datetime.datetime(time_init[0], time_init[1], time_init[2], 0, 0, 0), datetime.datetime(time_init[0], time_init[1]+1, 1, 0, 0, 0), ['mms3_mec_r_gse'], cdf=True)
        data_MMS4_MEC= cdas.get_data('sp_phys', 'MMS4_MEC_SRVY_L2_EPHTS04D',  datetime.datetime(time_init[0], time_init[1], time_init[2], 0, 0, 0), datetime.datetime(time_init[0], time_init[1]+1, 1, 0, 0, 0), ['mms4_mec_r_gse'], cdf=True)
    print('MEC data read')

    MMS1_Pi_time=np.array([])
    for t in range (len(MMS1_Pi.time)):
        MMS1_Pi_time=np.append(MMS1_Pi_time,datetime.datetime.utcfromtimestamp(MMS1_Pi.time[t].tolist()/1e9))
    MMS2_Pi_time=np.array([])
    for t in range (len(MMS2_Pi.time)):
        MMS2_Pi_time=np.append(MMS2_Pi_time,datetime.datetime.utcfromtimestamp(MMS2_Pi.time[t].tolist()/1e9))
    MMS3_Pi_time=np.array([])
    for t in range (len(MMS3_Pi.time)):
        MMS3_Pi_time=np.append(MMS3_Pi_time,datetime.datetime.utcfromtimestamp(MMS3_Pi.time[t].tolist()/1e9))
    MMS4_Pi_time=np.array([])
    for t in range (len(MMS4_Pi.time)):
        MMS4_Pi_time=np.append(MMS4_Pi_time,datetime.datetime.utcfromtimestamp(MMS4_Pi.time[t].tolist()/1e9))

    #_cutting time edges (je ne suis pas certaine de l'effet de ces lignes, peut-être qu'elles ne servent à rien)
    lista_time_arrays = [
        data_MMS1_FGM['Epoch_state'],data_MMS2_FGM['Epoch_state'],data_MMS3_FGM['Epoch_state'],data_MMS4_FGM['Epoch_state'],
        data_MMS1_FGM['Epoch'],data_MMS2_FGM['Epoch'],data_MMS3_FGM['Epoch'],data_MMS4_FGM['Epoch'],
        data_MMS1_FPI_i['Epoch'],data_MMS2_FPI_i['Epoch'],data_MMS3_FPI_i['Epoch'],data_MMS4_FPI_i['Epoch'],
        data_MMS1_FPI_e['Epoch'],data_MMS2_FPI_e['Epoch'],data_MMS3_FPI_e['Epoch'],data_MMS4_FPI_e['Epoch'],
        data_MMS1_EDP_E['mms1_edp_epoch_brst_l2'],data_MMS2_EDP_E['mms2_edp_epoch_brst_l2'],data_MMS3_EDP_E['mms3_edp_epoch_brst_l2'],data_MMS4_EDP_E['mms4_edp_epoch_brst_l2'],
        [datetime_init,datetime_end], MMS1_Pi_time,  MMS2_Pi_time, MMS3_Pi_time,  MMS4_Pi_time
    ]

    time_down, time_up = [], []
    for obj in lista_time_arrays:
        time = datetosec(obj,time_init)
        time_down.append(time[0])
        time_up.append(time[-1])

    time = [t for t in datetosec(data_MMS1_FGM['Epoch'],time_init) if sorted(time_down)[-1]<t<sorted(time_up)[0]]

    # Setting time as a regular grid (due to lack of precision, dt fluctuates over the whole interval)
    time = np.array(time)
    dt = 1 / frequency
    time = np.arange(time[0], time[-1], dt)

    #_interpolation linéaire
    #_position FGM
    MMS1_pos_x = interp1d(datetosec(data_MMS1_FGM['Epoch_state'],time_init), data_MMS1_FGM['mms1_fgm_r_gse_brst_l2'][:,0], kind='linear')(time)
    MMS2_pos_x = interp1d(datetosec(data_MMS2_FGM['Epoch_state'],time_init), data_MMS2_FGM['mms2_fgm_r_gse_brst_l2'][:,0], kind='linear')(time)
    MMS3_pos_x = interp1d(datetosec(data_MMS3_FGM['Epoch_state'],time_init), data_MMS3_FGM['mms3_fgm_r_gse_brst_l2'][:,0], kind='linear')(time)
    MMS4_pos_x = interp1d(datetosec(data_MMS4_FGM['Epoch_state'],time_init), data_MMS4_FGM['mms4_fgm_r_gse_brst_l2'][:,0], kind='linear')(time)

    MMS1_pos_y = interp1d(datetosec(data_MMS1_FGM['Epoch_state'],time_init), data_MMS1_FGM['mms1_fgm_r_gse_brst_l2'][:,1], kind='linear')(time)
    MMS2_pos_y = interp1d(datetosec(data_MMS2_FGM['Epoch_state'],time_init), data_MMS2_FGM['mms2_fgm_r_gse_brst_l2'][:,1], kind='linear')(time)
    MMS3_pos_y = interp1d(datetosec(data_MMS3_FGM['Epoch_state'],time_init), data_MMS3_FGM['mms3_fgm_r_gse_brst_l2'][:,1], kind='linear')(time)
    MMS4_pos_y = interp1d(datetosec(data_MMS4_FGM['Epoch_state'],time_init), data_MMS4_FGM['mms4_fgm_r_gse_brst_l2'][:,1], kind='linear')(time)

    MMS1_pos_z = interp1d(datetosec(data_MMS1_FGM['Epoch_state'],time_init), data_MMS1_FGM['mms1_fgm_r_gse_brst_l2'][:,2], kind='linear')(time)
    MMS2_pos_z = interp1d(datetosec(data_MMS2_FGM['Epoch_state'],time_init), data_MMS2_FGM['mms2_fgm_r_gse_brst_l2'][:,2], kind='linear')(time)
    MMS3_pos_z = interp1d(datetosec(data_MMS3_FGM['Epoch_state'],time_init), data_MMS3_FGM['mms3_fgm_r_gse_brst_l2'][:,2], kind='linear')(time)
    MMS4_pos_z = interp1d(datetosec(data_MMS4_FGM['Epoch_state'],time_init), data_MMS4_FGM['mms4_fgm_r_gse_brst_l2'][:,2], kind='linear')(time)

    #_mag field
    MMS1_b_x = interp1d(datetosec(data_MMS1_FGM['Epoch'],time_init), data_MMS1_FGM['mms1_fgm_b_gse_brst_l2'][:,0], kind='linear')(time)
    MMS2_b_x = interp1d(datetosec(data_MMS2_FGM['Epoch'],time_init), data_MMS2_FGM['mms2_fgm_b_gse_brst_l2'][:,0], kind='linear')(time)
    MMS3_b_x = interp1d(datetosec(data_MMS3_FGM['Epoch'],time_init), data_MMS3_FGM['mms3_fgm_b_gse_brst_l2'][:,0], kind='linear')(time)
    MMS4_b_x = interp1d(datetosec(data_MMS4_FGM['Epoch'],time_init), data_MMS4_FGM['mms4_fgm_b_gse_brst_l2'][:,0], kind='linear')(time)

    MMS1_b_y = interp1d(datetosec(data_MMS1_FGM['Epoch'],time_init), data_MMS1_FGM['mms1_fgm_b_gse_brst_l2'][:,1], kind='linear')(time)
    MMS2_b_y = interp1d(datetosec(data_MMS2_FGM['Epoch'],time_init), data_MMS2_FGM['mms2_fgm_b_gse_brst_l2'][:,1], kind='linear')(time)
    MMS3_b_y = interp1d(datetosec(data_MMS3_FGM['Epoch'],time_init), data_MMS3_FGM['mms3_fgm_b_gse_brst_l2'][:,1], kind='linear')(time)
    MMS4_b_y = interp1d(datetosec(data_MMS4_FGM['Epoch'],time_init), data_MMS4_FGM['mms4_fgm_b_gse_brst_l2'][:,1], kind='linear')(time)

    MMS1_b_z = interp1d(datetosec(data_MMS1_FGM['Epoch'],time_init), data_MMS1_FGM['mms1_fgm_b_gse_brst_l2'][:,2], kind='linear')(time)
    MMS2_b_z = interp1d(datetosec(data_MMS2_FGM['Epoch'],time_init), data_MMS2_FGM['mms2_fgm_b_gse_brst_l2'][:,2], kind='linear')(time)
    MMS3_b_z = interp1d(datetosec(data_MMS3_FGM['Epoch'],time_init), data_MMS3_FGM['mms3_fgm_b_gse_brst_l2'][:,2], kind='linear')(time)
    MMS4_b_z = interp1d(datetosec(data_MMS4_FGM['Epoch'],time_init), data_MMS4_FGM['mms4_fgm_b_gse_brst_l2'][:,2], kind='linear')(time)

    #_el field
    MMS1_e_x = interp1d(datetosec(data_MMS1_EDP_E['mms1_edp_epoch_brst_l2'],time_init), data_MMS1_EDP_E['mms1_edp_dce_gse_brst_l2'][:,0], kind='linear')(time)
    MMS2_e_x = interp1d(datetosec(data_MMS2_EDP_E['mms2_edp_epoch_brst_l2'],time_init), data_MMS2_EDP_E['mms2_edp_dce_gse_brst_l2'][:,0], kind='linear')(time)
    MMS3_e_x = interp1d(datetosec(data_MMS3_EDP_E['mms3_edp_epoch_brst_l2'],time_init), data_MMS3_EDP_E['mms3_edp_dce_gse_brst_l2'][:,0], kind='linear')(time)
    MMS4_e_x = interp1d(datetosec(data_MMS4_EDP_E['mms4_edp_epoch_brst_l2'],time_init), data_MMS4_EDP_E['mms4_edp_dce_gse_brst_l2'][:,0], kind='linear')(time)

    MMS1_e_y = interp1d(datetosec(data_MMS1_EDP_E['mms1_edp_epoch_brst_l2'],time_init), data_MMS1_EDP_E['mms1_edp_dce_gse_brst_l2'][:,1], kind='linear')(time)
    MMS2_e_y = interp1d(datetosec(data_MMS2_EDP_E['mms2_edp_epoch_brst_l2'],time_init), data_MMS2_EDP_E['mms2_edp_dce_gse_brst_l2'][:,1], kind='linear')(time)
    MMS3_e_y = interp1d(datetosec(data_MMS3_EDP_E['mms3_edp_epoch_brst_l2'],time_init), data_MMS3_EDP_E['mms3_edp_dce_gse_brst_l2'][:,1], kind='linear')(time)
    MMS4_e_y = interp1d(datetosec(data_MMS4_EDP_E['mms4_edp_epoch_brst_l2'],time_init), data_MMS4_EDP_E['mms4_edp_dce_gse_brst_l2'][:,1], kind='linear')(time)

    MMS1_e_z = interp1d(datetosec(data_MMS1_EDP_E['mms1_edp_epoch_brst_l2'],time_init), data_MMS1_EDP_E['mms1_edp_dce_gse_brst_l2'][:,2], kind='linear')(time)
    MMS2_e_z = interp1d(datetosec(data_MMS2_EDP_E['mms2_edp_epoch_brst_l2'],time_init), data_MMS2_EDP_E['mms2_edp_dce_gse_brst_l2'][:,2], kind='linear')(time)
    MMS3_e_z = interp1d(datetosec(data_MMS3_EDP_E['mms3_edp_epoch_brst_l2'],time_init), data_MMS3_EDP_E['mms3_edp_dce_gse_brst_l2'][:,2], kind='linear')(time)
    MMS4_e_z = interp1d(datetosec(data_MMS4_EDP_E['mms4_edp_epoch_brst_l2'],time_init), data_MMS4_EDP_E['mms4_edp_dce_gse_brst_l2'][:,2], kind='linear')(time)

    #interpolation cubiques ajoutées le 17 novembrer 2020
    #_ions densities
    MMS1_ni = interpolation_vn_GIU(time, datetosec(data_MMS1_FPI_i['Epoch'],time_init), data_MMS1_FPI_i['mms1_dis_numberdensity_brst'])
    MMS2_ni = interpolation_vn_GIU(time, datetosec(data_MMS2_FPI_i['Epoch'],time_init), data_MMS2_FPI_i['mms2_dis_numberdensity_brst'])
    MMS3_ni = interpolation_vn_GIU(time, datetosec(data_MMS3_FPI_i['Epoch'],time_init), data_MMS3_FPI_i['mms3_dis_numberdensity_brst'])
    MMS4_ni = interpolation_vn_GIU(time, datetosec(data_MMS4_FPI_i['Epoch'],time_init), data_MMS4_FPI_i['mms4_dis_numberdensity_brst'])

    #_eons densities
    MMS1_ne = interp1d(datetosec(data_MMS1_FPI_e['Epoch'],time_init), data_MMS1_FPI_e['mms1_des_numberdensity_brst'], kind='cubic')(time)
    MMS2_ne = interp1d(datetosec(data_MMS2_FPI_e['Epoch'],time_init), data_MMS2_FPI_e['mms2_des_numberdensity_brst'], kind='cubic')(time)
    MMS3_ne = interp1d(datetosec(data_MMS3_FPI_e['Epoch'],time_init), data_MMS3_FPI_e['mms3_des_numberdensity_brst'], kind='cubic')(time)
    MMS4_ne = interp1d(datetosec(data_MMS4_FPI_e['Epoch'],time_init), data_MMS4_FPI_e['mms4_des_numberdensity_brst'], kind='cubic')(time)

    #_Ui NEW VERSION
    MMS1_ui_x = interpolation_vn_GIU(time, datetosec(data_MMS1_FPI_i['Epoch'],time_init), data_MMS1_FPI_i['mms1_dis_bulkv_gse_brst'][:,0])*1e3
    MMS2_ui_x = interpolation_vn_GIU(time, datetosec(data_MMS2_FPI_i['Epoch'],time_init), data_MMS2_FPI_i['mms2_dis_bulkv_gse_brst'][:,0])*1e3
    MMS3_ui_x = interpolation_vn_GIU(time, datetosec(data_MMS3_FPI_i['Epoch'],time_init), data_MMS3_FPI_i['mms3_dis_bulkv_gse_brst'][:,0])*1e3
    MMS4_ui_x = interpolation_vn_GIU(time, datetosec(data_MMS4_FPI_i['Epoch'],time_init), data_MMS4_FPI_i['mms4_dis_bulkv_gse_brst'][:,0])*1e3



    MMS1_ui_y = interpolation_vn_GIU(time, datetosec(data_MMS1_FPI_i['Epoch'],time_init), data_MMS1_FPI_i['mms1_dis_bulkv_gse_brst'][:,1])*1e3
    MMS2_ui_y = interpolation_vn_GIU(time, datetosec(data_MMS2_FPI_i['Epoch'],time_init), data_MMS2_FPI_i['mms2_dis_bulkv_gse_brst'][:,1])*1e3
    MMS3_ui_y = interpolation_vn_GIU(time, datetosec(data_MMS3_FPI_i['Epoch'],time_init), data_MMS3_FPI_i['mms3_dis_bulkv_gse_brst'][:,1])*1e3
    MMS4_ui_y = interpolation_vn_GIU(time, datetosec(data_MMS4_FPI_i['Epoch'],time_init), data_MMS4_FPI_i['mms4_dis_bulkv_gse_brst'][:,1])*1e3

    MMS1_ui_z = interpolation_vn_GIU(time, datetosec(data_MMS1_FPI_i['Epoch'],time_init), data_MMS1_FPI_i['mms1_dis_bulkv_gse_brst'][:,2])*1e3
    MMS2_ui_z = interpolation_vn_GIU(time, datetosec(data_MMS2_FPI_i['Epoch'],time_init), data_MMS2_FPI_i['mms2_dis_bulkv_gse_brst'][:,2])*1e3
    MMS3_ui_z = interpolation_vn_GIU(time, datetosec(data_MMS3_FPI_i['Epoch'],time_init), data_MMS3_FPI_i['mms3_dis_bulkv_gse_brst'][:,2])*1e3
    MMS4_ui_z = interpolation_vn_GIU(time, datetosec(data_MMS4_FPI_i['Epoch'],time_init), data_MMS4_FPI_i['mms4_dis_bulkv_gse_brst'][:,2])*1e3

    #_Ue
    MMS1_ue_x = kmtom * interp1d(datetosec(data_MMS1_FPI_e['Epoch'],time_init), data_MMS1_FPI_e['mms1_des_bulkv_gse_brst'][:,0], kind='cubic')(time)
    MMS2_ue_x = kmtom * interp1d(datetosec(data_MMS2_FPI_e['Epoch'],time_init), data_MMS2_FPI_e['mms2_des_bulkv_gse_brst'][:,0], kind='cubic')(time)
    MMS3_ue_x = kmtom * interp1d(datetosec(data_MMS3_FPI_e['Epoch'],time_init), data_MMS3_FPI_e['mms3_des_bulkv_gse_brst'][:,0], kind='cubic')(time)
    MMS4_ue_x = kmtom * interp1d(datetosec(data_MMS4_FPI_e['Epoch'],time_init), data_MMS4_FPI_e['mms4_des_bulkv_gse_brst'][:,0], kind='cubic')(time)

    MMS1_ue_y = kmtom * interp1d(datetosec(data_MMS1_FPI_e['Epoch'],time_init), data_MMS1_FPI_e['mms1_des_bulkv_gse_brst'][:,1], kind='cubic')(time)
    MMS2_ue_y = kmtom * interp1d(datetosec(data_MMS2_FPI_e['Epoch'],time_init), data_MMS2_FPI_e['mms2_des_bulkv_gse_brst'][:,1], kind='cubic')(time)
    MMS3_ue_y = kmtom * interp1d(datetosec(data_MMS3_FPI_e['Epoch'],time_init), data_MMS3_FPI_e['mms3_des_bulkv_gse_brst'][:,1], kind='cubic')(time)
    MMS4_ue_y = kmtom * interp1d(datetosec(data_MMS4_FPI_e['Epoch'],time_init), data_MMS4_FPI_e['mms4_des_bulkv_gse_brst'][:,1], kind='cubic')(time)

    MMS1_ue_z = kmtom * interp1d(datetosec(data_MMS1_FPI_e['Epoch'],time_init), data_MMS1_FPI_e['mms1_des_bulkv_gse_brst'][:,2], kind='cubic')(time)
    MMS2_ue_z = kmtom * interp1d(datetosec(data_MMS2_FPI_e['Epoch'],time_init), data_MMS2_FPI_e['mms2_des_bulkv_gse_brst'][:,2], kind='cubic')(time)
    MMS3_ue_z = kmtom * interp1d(datetosec(data_MMS3_FPI_e['Epoch'],time_init), data_MMS3_FPI_e['mms3_des_bulkv_gse_brst'][:,2], kind='cubic')(time)
    MMS4_ue_z = kmtom * interp1d(datetosec(data_MMS4_FPI_e['Epoch'],time_init), data_MMS4_FPI_e['mms4_des_bulkv_gse_brst'][:,2], kind='cubic')(time)


    #_Te
    MMS1_Te_xx = interp1d(datetosec(data_MMS1_FPI_e['Epoch'],time_init), data_MMS1_FPI_e['mms1_des_temptensor_gse_brst'][:,0,0], kind='cubic')(time)
    MMS1_Te_xy = interp1d(datetosec(data_MMS1_FPI_e['Epoch'],time_init), data_MMS1_FPI_e['mms1_des_temptensor_gse_brst'][:,0,1], kind='cubic')(time)
    MMS1_Te_xz = interp1d(datetosec(data_MMS1_FPI_e['Epoch'],time_init), data_MMS1_FPI_e['mms1_des_temptensor_gse_brst'][:,0,2], kind='cubic')(time)
    MMS1_Te_yy = interp1d(datetosec(data_MMS1_FPI_e['Epoch'],time_init), data_MMS1_FPI_e['mms1_des_temptensor_gse_brst'][:,1,1], kind='cubic')(time)
    MMS1_Te_yz = interp1d(datetosec(data_MMS1_FPI_e['Epoch'],time_init), data_MMS1_FPI_e['mms1_des_temptensor_gse_brst'][:,1,2], kind='cubic')(time)
    MMS1_Te_zz = interp1d(datetosec(data_MMS1_FPI_e['Epoch'],time_init), data_MMS1_FPI_e['mms1_des_temptensor_gse_brst'][:,2,2], kind='cubic')(time)

    MMS2_Te_xx = interp1d(datetosec(data_MMS2_FPI_e['Epoch'],time_init), data_MMS2_FPI_e['mms2_des_temptensor_gse_brst'][:,0,0], kind='cubic')(time)
    MMS2_Te_xy = interp1d(datetosec(data_MMS2_FPI_e['Epoch'],time_init), data_MMS2_FPI_e['mms2_des_temptensor_gse_brst'][:,0,1], kind='cubic')(time)
    MMS2_Te_xz = interp1d(datetosec(data_MMS2_FPI_e['Epoch'],time_init), data_MMS2_FPI_e['mms2_des_temptensor_gse_brst'][:,0,2], kind='cubic')(time)
    MMS2_Te_yy = interp1d(datetosec(data_MMS2_FPI_e['Epoch'],time_init), data_MMS2_FPI_e['mms2_des_temptensor_gse_brst'][:,1,1], kind='cubic')(time)
    MMS2_Te_yz = interp1d(datetosec(data_MMS2_FPI_e['Epoch'],time_init), data_MMS2_FPI_e['mms2_des_temptensor_gse_brst'][:,1,2], kind='cubic')(time)
    MMS2_Te_zz = interp1d(datetosec(data_MMS2_FPI_e['Epoch'],time_init), data_MMS2_FPI_e['mms2_des_temptensor_gse_brst'][:,2,2], kind='cubic')(time)

    MMS3_Te_xx = interp1d(datetosec(data_MMS3_FPI_e['Epoch'],time_init), data_MMS3_FPI_e['mms3_des_temptensor_gse_brst'][:,0,0], kind='cubic')(time)
    MMS3_Te_xy = interp1d(datetosec(data_MMS3_FPI_e['Epoch'],time_init), data_MMS3_FPI_e['mms3_des_temptensor_gse_brst'][:,0,1], kind='cubic')(time)
    MMS3_Te_xz = interp1d(datetosec(data_MMS3_FPI_e['Epoch'],time_init), data_MMS3_FPI_e['mms3_des_temptensor_gse_brst'][:,0,2], kind='cubic')(time)
    MMS3_Te_yy = interp1d(datetosec(data_MMS3_FPI_e['Epoch'],time_init), data_MMS3_FPI_e['mms3_des_temptensor_gse_brst'][:,1,1], kind='cubic')(time)
    MMS3_Te_yz = interp1d(datetosec(data_MMS3_FPI_e['Epoch'],time_init), data_MMS3_FPI_e['mms3_des_temptensor_gse_brst'][:,1,2], kind='cubic')(time)
    MMS3_Te_zz = interp1d(datetosec(data_MMS3_FPI_e['Epoch'],time_init), data_MMS3_FPI_e['mms3_des_temptensor_gse_brst'][:,2,2], kind='cubic')(time)

    MMS4_Te_xx = interp1d(datetosec(data_MMS4_FPI_e['Epoch'],time_init), data_MMS4_FPI_e['mms4_des_temptensor_gse_brst'][:,0,0], kind='cubic')(time)
    MMS4_Te_xy = interp1d(datetosec(data_MMS4_FPI_e['Epoch'],time_init), data_MMS4_FPI_e['mms4_des_temptensor_gse_brst'][:,0,1], kind='cubic')(time)
    MMS4_Te_xz = interp1d(datetosec(data_MMS4_FPI_e['Epoch'],time_init), data_MMS4_FPI_e['mms4_des_temptensor_gse_brst'][:,0,2], kind='cubic')(time)
    MMS4_Te_yy = interp1d(datetosec(data_MMS4_FPI_e['Epoch'],time_init), data_MMS4_FPI_e['mms4_des_temptensor_gse_brst'][:,1,1], kind='cubic')(time)
    MMS4_Te_yz = interp1d(datetosec(data_MMS4_FPI_e['Epoch'],time_init), data_MMS4_FPI_e['mms4_des_temptensor_gse_brst'][:,1,2], kind='cubic')(time)
    MMS4_Te_zz = interp1d(datetosec(data_MMS4_FPI_e['Epoch'],time_init), data_MMS4_FPI_e['mms4_des_temptensor_gse_brst'][:,2,2], kind='cubic')(time)

    #ajout LR novembre 2020
    #_Ions spectrograms
    MMS1_Spectr_i = np.array([interp1d(datetosec(data_MMS1_FPI_i['Epoch'],time_init), data_MMS1_FPI_i['mms1_dis_energyspectr_omni_brst'][:,i], kind='cubic')(time) for i in np.arange(32)])
    MMS2_Spectr_i = np.array([interp1d(datetosec(data_MMS2_FPI_i['Epoch'],time_init), data_MMS2_FPI_i['mms2_dis_energyspectr_omni_brst'][:,i], kind='cubic')(time) for i in np.arange(32)])
    MMS3_Spectr_i = np.array([interp1d(datetosec(data_MMS3_FPI_i['Epoch'],time_init), data_MMS3_FPI_i['mms3_dis_energyspectr_omni_brst'][:,i], kind='cubic')(time) for i in np.arange(32)])
    MMS4_Spectr_i = np.array([interp1d(datetosec(data_MMS4_FPI_i['Epoch'],time_init), data_MMS4_FPI_i['mms4_dis_energyspectr_omni_brst'][:,i], kind='cubic')(time) for i in np.arange(32)])

     #_E-ons spectrograms
    MMS1_Spectr_e = np.array([interp1d(datetosec(data_MMS1_FPI_e['Epoch'],time_init), data_MMS1_FPI_e['mms1_des_energyspectr_omni_brst'][:,i], kind='cubic')(time) for i in np.arange(32)])
    MMS2_Spectr_e = np.array([interp1d(datetosec(data_MMS2_FPI_e['Epoch'],time_init), data_MMS2_FPI_e['mms2_des_energyspectr_omni_brst'][:,i], kind='cubic')(time) for i in np.arange(32)])
    MMS3_Spectr_e = np.array([interp1d(datetosec(data_MMS3_FPI_e['Epoch'],time_init), data_MMS3_FPI_e['mms3_des_energyspectr_omni_brst'][:,i], kind='cubic')(time) for i in np.arange(32)])
    MMS4_Spectr_e = np.array([interp1d(datetosec(data_MMS4_FPI_e['Epoch'],time_init), data_MMS4_FPI_e['mms4_des_energyspectr_omni_brst'][:,i], kind='cubic')(time) for i in np.arange(32)])


    #interpolation quadratique des positions MEC
    MMS1_pos_x_mec = interp1d(datetosec(data_MMS1_MEC['Epoch'],time_init), data_MMS1_MEC['mms1_mec_r_gse'][:,0], kind='quadratic')(time)
    MMS2_pos_x_mec = interp1d(datetosec(data_MMS2_MEC['Epoch'],time_init), data_MMS2_MEC['mms2_mec_r_gse'][:,0], kind='quadratic')(time)
    MMS3_pos_x_mec = interp1d(datetosec(data_MMS3_MEC['Epoch'],time_init), data_MMS3_MEC['mms3_mec_r_gse'][:,0], kind='quadratic')(time)
    MMS4_pos_x_mec = interp1d(datetosec(data_MMS4_MEC['Epoch'],time_init), data_MMS4_MEC['mms4_mec_r_gse'][:,0], kind='quadratic')(time)

    MMS1_pos_y_mec = interp1d(datetosec(data_MMS1_MEC['Epoch'],time_init), data_MMS1_MEC['mms1_mec_r_gse'][:,1], kind='quadratic')(time)
    MMS2_pos_y_mec = interp1d(datetosec(data_MMS2_MEC['Epoch'],time_init), data_MMS2_MEC['mms2_mec_r_gse'][:,1], kind='quadratic')(time)
    MMS3_pos_y_mec = interp1d(datetosec(data_MMS3_MEC['Epoch'],time_init), data_MMS3_MEC['mms3_mec_r_gse'][:,1], kind='quadratic')(time)
    MMS4_pos_y_mec = interp1d(datetosec(data_MMS4_MEC['Epoch'],time_init), data_MMS4_MEC['mms4_mec_r_gse'][:,1], kind='quadratic')(time)

    MMS1_pos_z_mec = interp1d(datetosec(data_MMS1_MEC['Epoch'],time_init), data_MMS1_MEC['mms1_mec_r_gse'][:,2], kind='quadratic')(time)
    MMS2_pos_z_mec = interp1d(datetosec(data_MMS2_MEC['Epoch'],time_init), data_MMS2_MEC['mms2_mec_r_gse'][:,2], kind='quadratic')(time)
    MMS3_pos_z_mec = interp1d(datetosec(data_MMS3_MEC['Epoch'],time_init), data_MMS3_MEC['mms3_mec_r_gse'][:,2], kind='quadratic')(time)
    MMS4_pos_z_mec = interp1d(datetosec(data_MMS4_MEC['Epoch'],time_init), data_MMS4_MEC['mms4_mec_r_gse'][:,2], kind='quadratic')(time)

    MMS1_Pi_interp=interpolation_pressure_GIU(time, datetosec(MMS1_Pi_time,time_init), MMS1_Pi.values)
    MMS2_Pi_interp=interpolation_pressure_GIU(time, datetosec(MMS2_Pi_time,time_init), MMS2_Pi.values)
    MMS3_Pi_interp=interpolation_pressure_GIU(time, datetosec(MMS3_Pi_time,time_init), MMS3_Pi.values)
    MMS4_Pi_interp=interpolation_pressure_GIU(time, datetosec(MMS4_Pi_time,time_init), MMS4_Pi.values)

    # ecriture finale dans les fichiers
    #_Computation of Js

    MMS1_J_x = cm3tom3*qe*(MMS1_ni*MMS1_ui_x-MMS1_ne*MMS1_ue_x)
    MMS2_J_x = cm3tom3*qe*(MMS2_ni*MMS2_ui_x-MMS2_ne*MMS2_ue_x)
    MMS3_J_x = cm3tom3*qe*(MMS3_ni*MMS3_ui_x-MMS3_ne*MMS3_ue_x)
    MMS4_J_x = cm3tom3*qe*(MMS4_ni*MMS4_ui_x-MMS4_ne*MMS4_ue_x)

    MMS1_J_y = cm3tom3*qe*(MMS1_ni*MMS1_ui_y-MMS1_ne*MMS1_ue_y)
    MMS2_J_y = cm3tom3*qe*(MMS2_ni*MMS2_ui_y-MMS2_ne*MMS2_ue_y)
    MMS3_J_y = cm3tom3*qe*(MMS3_ni*MMS3_ui_y-MMS3_ne*MMS3_ue_y)
    MMS4_J_y = cm3tom3*qe*(MMS4_ni*MMS4_ui_y-MMS4_ne*MMS4_ue_y)

    MMS1_J_z = cm3tom3*qe*(MMS1_ni*MMS1_ui_z-MMS1_ne*MMS1_ue_z)
    MMS2_J_z = cm3tom3*qe*(MMS2_ni*MMS2_ui_z-MMS2_ne*MMS2_ue_z)
    MMS3_J_z = cm3tom3*qe*(MMS3_ni*MMS3_ui_z-MMS3_ne*MMS3_ue_z)
    MMS4_J_z = cm3tom3*qe*(MMS4_ni*MMS4_ui_z-MMS4_ne*MMS4_ue_z)

    #No temperature version
    new_organized_array = np.array([
    [time, MMS1_pos_x_mec, MMS1_pos_y_mec, MMS1_pos_z_mec, MMS1_b_x, MMS1_b_y, MMS1_b_z, MMS1_ni, MMS1_ui_x, MMS1_ui_y, MMS1_ui_z, MMS1_ne, MMS1_ue_x, MMS1_ue_y, MMS1_ue_z, MMS1_Pi_interp[0,0], MMS1_Pi_interp[1,1], MMS1_Pi_interp[2,2], MMS1_Pi_interp[0,1], MMS1_Pi_interp[0,2], MMS1_Pi_interp[1,2], MMS1_Te_xx, MMS1_Te_yy, MMS1_Te_zz, MMS1_Te_xy, MMS1_Te_xz, MMS1_Te_yz, MMS1_e_x, MMS1_e_y, MMS1_e_z, MMS1_J_x, MMS1_J_y, MMS1_J_z],
    [time, MMS2_pos_x_mec, MMS2_pos_y_mec, MMS2_pos_z_mec, MMS2_b_x, MMS2_b_y, MMS2_b_z, MMS2_ni, MMS2_ui_x, MMS2_ui_y, MMS2_ui_z, MMS2_ne, MMS2_ue_x, MMS2_ue_y, MMS2_ue_z, MMS2_Pi_interp[0,0], MMS2_Pi_interp[1,1], MMS2_Pi_interp[2,2], MMS2_Pi_interp[0,1], MMS2_Pi_interp[0,2], MMS2_Pi_interp[1,2], MMS2_Te_xx, MMS2_Te_yy, MMS2_Te_zz, MMS2_Te_xy, MMS2_Te_xz, MMS2_Te_yz, MMS2_e_x, MMS2_e_y, MMS2_e_z, MMS2_J_x, MMS2_J_y, MMS2_J_z],
    [time, MMS3_pos_x_mec, MMS3_pos_y_mec, MMS3_pos_z_mec, MMS3_b_x, MMS3_b_y, MMS3_b_z, MMS3_ni, MMS3_ui_x, MMS3_ui_y, MMS3_ui_z, MMS3_ne, MMS3_ue_x, MMS3_ue_y, MMS3_ue_z, MMS3_Pi_interp[0,0], MMS3_Pi_interp[1,1], MMS3_Pi_interp[2,2], MMS3_Pi_interp[0,1], MMS3_Pi_interp[0,2], MMS3_Pi_interp[1,2], MMS3_Te_xx, MMS3_Te_yy, MMS3_Te_zz, MMS3_Te_xy, MMS3_Te_xz, MMS3_Te_yz, MMS3_e_x, MMS3_e_y, MMS3_e_z, MMS3_J_x, MMS3_J_y, MMS3_J_z],
    [time, MMS4_pos_x_mec, MMS4_pos_y_mec, MMS4_pos_z_mec, MMS4_b_x, MMS4_b_y, MMS4_b_z, MMS4_ni, MMS4_ui_x, MMS4_ui_y, MMS4_ui_z, MMS4_ne, MMS4_ue_x, MMS4_ue_y, MMS4_ue_z, MMS4_Pi_interp[0,0], MMS4_Pi_interp[1,1], MMS4_Pi_interp[2,2], MMS4_Pi_interp[0,1], MMS4_Pi_interp[0,2], MMS4_Pi_interp[1,2], MMS4_Te_xx, MMS4_Te_yy, MMS4_Te_zz, MMS4_Te_xy, MMS4_Te_xz, MMS4_Te_yz, MMS4_e_x, MMS4_e_y, MMS4_e_z, MMS4_J_x, MMS4_J_y, MMS4_J_z]])

    #_building the filenames
    dt = datetime_end-datetime_init
    dt = dt.seconds
    ymd = str('{:04d}'.format(time_init[0])) + str('{:02d}'.format(time_init[1])) + str('{:02d}'.format(time_init[2]))
    hms = str('{:02d}'.format(time_init[3])) + str('{:02d}'.format(time_init[4])) + str('{:02d}'.format(time_init[5]))

    #_writing files
    titles = ' time(s)  x(km)  y(km)  z(km)  bx(nT)  by(nT)  bz(nT)  ni(cm^-3)  vix(m/s)  viy(m/s)  viz(m/s)  ne(cm^-3)  vex(m/s)  vey(m/s)  vez(m/s)  Pixx(nPa)  Piyy(nPa)  Pizz(nPa)  Pixy(nPa)  Pixz(nPa)  Piyz(nPa)  Texx(eV)  Teyy(eV)  Tezz(eV)  Texy(eV)  Texz(eV)  Teyz(eV)  Ex(mV/m)  Ey(mV/m)  Ez(mV/m)  Jx(A/m^2)  Jy(A/m^2)  Jz(A/m^2) \n'


    #GIULIO qua devi mettere nome cartella
    for k in np.arange(4):
        print('# ==> Writing the txt file for ' + 'mms' + str(k + 1) + ':   ', end='')
        filename = '../NEWDATA/'+ ymd+'/'+ hms + '_' + str(dt) +'/mms' + str(k + 1) + '_GSE_' + ymd + '_' + hms + '_' + str(dt) + '.txt'
        print(filename)
        if not os.path.exists(os.path.dirname(filename)):
            try:
                os.makedirs(os.path.dirname(filename))
            except OSError as exc: # Guard against race condition
                if exc.errno != errno.EEXIST:
                    raise
        fw = open(filename, "w")
        fw.write(titles)

        for i in np.arange(len(time)):
            stringa = '        '.join([str(obj) for obj in new_organized_array[k,:,i]]) + ' \n' 
            fw.write(stringa)  
        fw.close()

    print("Files written on disk.") 

/tmp/ipykernel_44814/4008859888.py:2: DeprecationWarning: `np.str` is a deprecated alias for the builtin `str`. To silence this warning, use `str` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.str_` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  directory_array=np.loadtxt("/home/ballerini/Desktop/NEWstudy/data/txt_tables/folder.txt", dtype=np.str)


20160101 212210_17
[2015, 10, 16, 13, 5, 24]
2015-10-16 13:05:24 2015-10-16 13:06:07
100% [........................................................] 457212 / 457212


/home/ballerini/venv/lib/python3.10/site-packages/spacepy/time.py:2367: UserWarning: Leapseconds may be out of date. Use spacepy.toolbox.update(leapsecs=True)
  warnings.warn('Leapseconds may be out of date.'


100% [........................................................] 457003 / 457003
100% [........................................................] 457174 / 457174
100% [........................................................] 459868 / 459868
FGM data read
100% [........................................................] 251502 / 251502
100% [........................................................] 251899 / 251899
100% [........................................................] 251900 / 251900
100% [........................................................] 251745 / 251745
100% [......................................................] 1007287 / 1007287
100% [......................................................] 1005253 / 1005253
100% [......................................................] 1007541 / 1007541
100% [......................................................] 1009988 / 1009988
FPI data read
100% [....................................................] 22193287 / 22193287
100% [......

In [8]:
print(data_MMS1_MEC['mms1_mec_r_gse'][0,:])
print(data_MMS1_FGM['mms1_fgm_r_gse_brst_l2'][0,:])


[-8398.05967979 35130.90724655 -3922.94665774]
[45456.54   49316.016   -601.4955 67072.555 ]


In [42]:
MMS1_Pi_time=np.array([])
for t in range (len(MMS1_Pi.time)):
    MMS1_Pi_time=np.append(MMS1_Pi_time,datetime.datetime.utcfromtimestamp(MMS1_Pi.time[t].tolist()/1e9))
MMS2_Pi_time=np.array([])
for t in range (len(MMS2_Pi.time)):
    MMS2_Pi_time=np.append(MMS2_Pi_time,datetime.datetime.utcfromtimestamp(MMS2_Pi.time[t].tolist()/1e9))
MMS3_Pi_time=np.array([])
for t in range (len(MMS3_Pi.time)):
    MMS3_Pi_time=np.append(MMS3_Pi_time,datetime.datetime.utcfromtimestamp(MMS3_Pi.time[t].tolist()/1e9))
MMS4_Pi_time=np.array([])
for t in range (len(MMS4_Pi.time)):
    MMS4_Pi_time=np.append(MMS4_Pi_time,datetime.datetime.utcfromtimestamp(MMS4_Pi.time[t].tolist()/1e9))

#_cutting time edges (je ne suis pas certaine de l'effet de ces lignes, peut-être qu'elles ne servent à rien)
lista_time_arrays = [
    data_MMS1_FGM['Epoch_state'],data_MMS2_FGM['Epoch_state'],data_MMS3_FGM['Epoch_state'],data_MMS4_FGM['Epoch_state'],
    data_MMS1_FGM['Epoch'],data_MMS2_FGM['Epoch'],data_MMS3_FGM['Epoch'],data_MMS4_FGM['Epoch'],
    data_MMS1_FPI_i['Epoch'],data_MMS2_FPI_i['Epoch'],data_MMS3_FPI_i['Epoch'],data_MMS4_FPI_i['Epoch'],
    data_MMS1_FPI_e['Epoch'],data_MMS2_FPI_e['Epoch'],data_MMS3_FPI_e['Epoch'],data_MMS4_FPI_e['Epoch'],
    data_MMS1_EDP_E['mms1_edp_epoch_brst_l2'],data_MMS2_EDP_E['mms2_edp_epoch_brst_l2'],data_MMS3_EDP_E['mms3_edp_epoch_brst_l2'],data_MMS4_EDP_E['mms4_edp_epoch_brst_l2'],
    [datetime_init,datetime_end], MMS1_Pi_time,  MMS2_Pi_time, MMS3_Pi_time,  MMS4_Pi_time
]

time_down, time_up = [], []
for obj in lista_time_arrays:
    print(obj, time_init)
    time = datetosec(obj,time_init)
    time_down.append(time[0])
    time_up.append(time[-1])


[datetime.datetime(2016, 1, 1, 21, 21, 24, 284000)
 datetime.datetime(2016, 1, 1, 21, 21, 54, 284000)
 datetime.datetime(2016, 1, 1, 21, 22, 24, 284000)
 datetime.datetime(2016, 1, 1, 21, 22, 54, 284000)
 datetime.datetime(2016, 1, 1, 21, 23, 24, 284000)
 datetime.datetime(2016, 1, 1, 21, 22, 24, 284000)
 datetime.datetime(2016, 1, 1, 21, 22, 54, 284000)
 datetime.datetime(2016, 1, 1, 21, 23, 24, 284000)] [2016, 1, 1, 21, 22, 10, 17]


ValueError: too many values to unpack (expected 6)